#  word2vec, doc2vec и fasttext 


1. word2vec – векторное представление слова
2. как сделать вектор документа?
    * усреднить все вектора слов
    * усреднить все вектора слов с $tf-idf$ весами
    * doc2vec
3. fasttext – векторное представление $n$-грам


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
from sklearn.metrics import *
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
random.seed(1228)

%matplotlib inline

Загружаем лемматизированные статьи без стоп-слов и создаем массив текстов

In [3]:
from pymystem3 import Mystem
import re


m = Mystem()


regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""



def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "


In [ ]:
df_neg = pd.read_csv('datasets/nlp/negative.csv', sep=';', header=None, usecols=[3])
df_pos = pd.read_csv('datasets/nlp/positive.csv', sep=';', header=None, usecols=[3])
df_neg['sent'] = 'neg'
df_pos['sent'] = 'pos'
df = pd.concat([df_neg, df_pos])
df.columns = ['text', 'sent']
df.text = df.text.apply(words_only)
df.text = df.text.apply(lemmatize)

In [ ]:
df.head()

In [ ]:
texts = [df.text.iloc[i].split() for i in range(len(df))]

## Обучение модели в gensim

In [ ]:
text[0]

In [ ]:
%%time
from gensim.models import Word2Vec
model = Word2Vec(text, size=100, window=5, min_count=5, workers=6)
model.save('sent_w2v.model')

Загружаем обученную модель

In [ ]:
from gensim.models import Word
model.load('sent_w2v.model')

In [ ]:
model.most_similar(':)')

In [ ]:
model.most_similar(positive=['хорошо', 'плохой'], negative=['хороший'])

In [ ]:
model.doesnt_match('борщ сметана макароны пирожок консомэ кошка'.split())

Визуализация пространства слов

In [ ]:
top_words = []
from nltk import FreqDist
fd = FreqDist()
for text in texts:
    fd.update(text)
for i in fd.most_common(1000):
    top_words.append(i[0])
    
print(top_words)

In [ ]:
top_words_vec = model[top_words]

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_componets=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools='pan, wheel_zoom, reset, save,
           toolbar_location='above',
           title='word2vec T-SNE for most common words')
source = ColumnDataSource(data=dict(x1=top_word_tsne[:, 0],
                                    x2=top_word_tsne[:, 1],
                                    names=top_words
                                   ))
p.scatter(x='x1', y='x2', size=8, source=source)

labels = LabelSet(x='x1', y='x2', text='names', y_offset=6,
                  text_font_size='8pt', text_color='#555555',
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Кластеризация слов

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

dist = 1 - cosine_similarity(top_woprds_vec)

In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram

linkage_matrix = ward(dist)

fig, ax = plt.subplots(figsize=(10, 100))
ax = dendrogram(linkage_matrix, orientation='right', labels=top_words);

plt.tick_params(\
                axis='x', # changes apply to x-axis
                which='both', # both major and minor ticks are affected
                bottom='off', # ticks along the bottom edge are off
                top='off', # ticks along the top edge are off
                label_bottom='off')
plt.tight_layout()
plt.savefig('w2_clusters.pgn', dpi=200) #save figure as ward_clusters

## Классификация текстов 

По мотивам http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

In [ ]:
X = df.text.tolist()
y = df.sent.tolist()

X, y = np.array(X), np.array(y)

X_train, y_train, X_train, y_train = train_test_split(X, y, text_size=.33)
print(f'total train examples {len(y_train)}')
print(f'total test examples {len(y_test)}')

In [ ]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionalityas all the other vectors
        self.dim = len(w2v.popitem()[1])
    
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                   or [np.zeros(self.dim)], axis=0)
            for word in X
        ])

In [ ]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(w2v.popitem()[1])
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(x)
        max_idf = max(tfidf.idf_)
        self.word2weight = datadict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
        
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] * self.word2weight[w]
                    for w in words if w in self.word2vec] or
                   [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [ ]:
w2v = dict(zip(model.wvindex2word, model.wv.syn0))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

rfc_w2v = Pipeline([
    ('word2vec vectorizer', MeanEmbeddingVectorizer(w2v)),
    ('extra trees', RandomForestClassifier(n_estimators=20))])

rfc_w2v_tfidf = Pipeline([
    ('word2vec vectorizer', TfidfEmbeddingVectorizer(w2v)),
    ('extra trees', RandomForestClassifier(n_estimators=20))])

In [ ]:
rfc_w2v.fit(X_train, y_train)
pred = rfc_w2v.predict(X_test)